# Простой пример модели детектирования акустических событий.

## Импортируем необходимые библиотеки

In [ ]:
import numpy as np
from librosa.feature import melspectrogram
from librosa.core import load as load_wav
from librosa.display import specshow
from librosa import power_to_db
import os
import matplotlib.pyplot as plt

## Скачивание данных и подготовка к обучению

Скачать архив с данными можно, перейдя по ссылке. Для скачивания потребуется авторизация на kaggle.com. 

Затем необходимо положить данные для трейна и теста в папки data/audio_train и data/audio_test соответственно, так же в корень папки необходимо положить файл train.csv, который также можно найти в архиве. 

Далее задаются различные параметры для извлечения фичей из файлов.

In [ ]:
train_folder = './data/audio_train'
test_folder = './data/audio_train'


sample_rate = 8000
fft_size = 1024
overlap = 4
hop = fft_size // overlap
mels = 64

## Визуализация фичей, используемых при обучении, для одного файла.

In [ ]:
wav_data, sr = load_wav(os.path.join(train_folder, 'aaf6167e.wav'), sr=sample_rate)

mel_spec = melspectrogram(wav_data, n_fft=fft_size, hop_length=hop, n_mels=mels, 
                          fmax=sample_rate//2)

plt.figure(figsize=(10, 4))
specshow(mel_spec, y_axis='mel',
         fmax=sample_rate//2, x_axis='time')
plt.colorbar(format='%+2.0f дБ')
plt.title('Мелспектрограмма скрипки')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
specshow(power_to_db(mel_spec, ref=np.max), y_axis='mel',
         fmax=sample_rate//2, x_axis='time')
plt.colorbar(format='%+2.0f дБ')
plt.title('Логарифм мелспектрограммы скрипки')
plt.tight_layout()
plt.show()

## Извлечение признаков


Для ускорения вычислений нам будет полезно вычислить всем мелспектрограммы заранее и сохранить их. Выполнение данной ячейки займет некоторое время, так как процесс вычисления признаков дорогой. Размер полученных файлов будет приблизительно 1 гб каждый (при исходных значениях параметров).